In [1]:
import sqlite3

In [41]:
romanian_texts = {}
moldavian_texts = {}

conn = sqlite3.connect('news.db')
c = conn.cursor()

c.execute('SELECT * FROM romania')
rows = c.fetchall()
for row in rows:
    if row[4] not in romanian_texts:
        romanian_texts[row[4]] = []
    romanian_texts[row[4]].append(row[5])
    
c.execute('SELECT * FROM moldova WHERE newspaper != "zugo"')
rows = c.fetchall()
for row in rows:
    if row[4] not in moldavian_texts:
        moldavian_texts[row[4]] = []
    moldavian_texts[row[4]].append(row[5])

conn.close()

In [43]:
all_texts = {"romana": [], "moldova": []}

for key in romanian_texts:
    all_texts["romana"].extend(romanian_texts[key])

    
for key in moldavian_texts:
    all_texts["moldova"].extend(moldavian_texts[key])

In [4]:
!python -m spacy download ro_core_news_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 MB 4.7 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('ro_core_news_md')


In [44]:
import spacy

nlp = spacy.load('ro_core_news_md')

all_texts["romana"] = [nlp(text) for text in all_texts["romana"]]
all_texts["moldova"] = [nlp(text) for text in all_texts["moldova"]]


# Dependency tree
all_dependencies = {"romana": [], "moldova": []}
all_pos = {"romana": [], "moldova": []}

for text in all_texts["romana"]:
    for sent in text.sents:
        token_dep_list = []
        token_pos_list = []
        for token in sent:
            token_dep_list.append(token.dep_)
            token_pos_list.append(token.pos_)
        all_dependencies["romana"].append(token_dep_list)
        all_pos["romana"].append(token_pos_list)

for text in all_texts["moldova"]:
    for sent in text.sents:
        token_dep_list = []
        token_pos_list = []
        for token in sent:
            token_dep_list.append(token.dep_)
            token_pos_list.append(token.pos_)
        all_dependencies["moldova"].append(token_dep_list)
        all_pos["moldova"].append(token_pos_list)
    

In [45]:
# Most common syntactic dependencies
from collections import Counter

# most_common_dependencies = {"romana": [], "moldova": []}

# deps = {}
# for sent in all_dependencies["romana"]:
#     for dep in sent:
#         if dep not in deps:
#             deps[dep] = 2**len(deps)
        
# for sent in all_dependencies["moldova"]:
#     for dep in sent:
#         if dep not in deps:
#             deps[dep] = 2**len(deps)

# Romana
sentence_numbers = {}        
sentences = {}
dep_sentences = {}
for i in range(len(all_pos["romana"])):
    sent = all_pos["romana"][i]
    sentence = [tag for tag in sent]
    sum = []
    for tag in sentence:
        sum.append(str(tag))
    if(len(sum) < 3):
        continue
    sum.sort()
    sum = ' '.join(sum)
    if sum not in sentence_numbers:
        sentence_numbers[sum] = 0
    else:
        sentence_numbers[sum] += 1
        
    if sum not in sentences:
        sentences[sum] = sent
        dep_sentences[sum] = all_dependencies["romana"][i]
    
print("Romana")  
sorted_sent_numbers = sorted(sentence_numbers, reverse=True, key=lambda x: sentence_numbers[x])
mc_romana = sorted_sent_numbers[:3]

for mc in mc_romana:
    print(mc, sentence_numbers[mc])

for mc in mc_romana:
    dep_sent = dep_sentences[mc]
    spacy.displacy.render(nlp(' '.join(dep_sent)))

# Moldova
sentence_numbers_md = {}        
sentences_md = {}
dep_sentences_md = {}
for i in range(len(all_pos["moldova"])):
    sent = all_pos["moldova"][i]
    sentence = [tag for tag in sent]
    sum = []
    for tag in sentence:
        sum.append(str(tag))
    if(len(sum) < 3):
        continue
    sum.sort()
    sum = ' '.join(sum)
    if sum not in sentence_numbers_md:
        sentence_numbers_md[sum] = 0
    else:
        sentence_numbers_md[sum] += 1
        
    if sum not in sentences_md:
        sentences_md[sum] = sent
        dep_sentences_md[sum] = all_dependencies["moldova"][i]
    
print("Moldova")  
sorted_sent_numbers = sorted(sentence_numbers_md, reverse=True, key=lambda x: sentence_numbers_md[x])
mc_moldova = sorted_sent_numbers[:3]

for mc in mc_moldova:
    print(mc, sentence_numbers_md[mc])

for mc in mc_moldova:
    dep_sent = dep_sentences_md[mc]
    spacy.displacy.render(nlp(' '.join(dep_sent)))


Romana
ADJ ADP ADP ADP ADP AUX DET NOUN NOUN NOUN PRON PRON PUNCT PUNCT PUNCT SPACE VERB VERB VERB 517
ADJ ADP ADP ADV ADV AUX DET NOUN PROPN PUNCT SPACE 166
ADP ADV ADV NOUN NOUN PRON PUNCT PUNCT SPACE VERB 21


Moldova
ADP ADP ADV AUX NOUN NOUN PRON PUNCT 121
ADP ADP AUX NOUN NOUN NOUN NOUN PRON PUNCT PUNCT 115
ADJ ADP ADP ADP ADP ADP ADV ADV AUX AUX DET DET NOUN NOUN NOUN NOUN NUM PRON PROPN PUNCT PUNCT VERB 114


In [47]:
print("RO ---> MD")
for mc in mc_romana:
    md_count = 0
    if mc in sentence_numbers_md:
        md_count = sentence_numbers_md[mc]
    print(f"{mc} - RO:{sentence_numbers[mc]} - MD:{md_count}")

print("\n")

print("MD ---> RO")
for mc in mc_moldova:
    ro_count = 0
    if mc in sentence_numbers:
        ro_count = sentence_numbers[mc]
    print(f"{mc} - RO:{ro_count} - MD:{sentence_numbers_md[mc]}")

RO ---> MD
ADJ ADP ADP ADP ADP AUX DET NOUN NOUN NOUN PRON PRON PUNCT PUNCT PUNCT SPACE VERB VERB VERB - RO:517 - MD:0
ADJ ADP ADP ADV ADV AUX DET NOUN PROPN PUNCT SPACE - RO:166 - MD:0
ADP ADV ADV NOUN NOUN PRON PUNCT PUNCT SPACE VERB - RO:21 - MD:0


MD ---> RO
ADP ADP ADV AUX NOUN NOUN PRON PUNCT - RO:0 - MD:121
ADP ADP AUX NOUN NOUN NOUN NOUN PRON PUNCT PUNCT - RO:0 - MD:115
ADJ ADP ADP ADP ADP ADP ADV ADV AUX AUX DET DET NOUN NOUN NOUN NOUN NUM PRON PROPN PUNCT PUNCT VERB - RO:0 - MD:114


In [48]:
ro_md = {}
for sent in sentence_numbers:
    if sent in sentence_numbers_md:
        ro_md[sent] = {'ro': sentence_numbers[sent], 'md': sentence_numbers_md[sent]}
        
print(f"Ro-Md common: {len(ro_md)}")
print(f"Ro common percentage: {len(ro_md) / len(sentence_numbers) * 100:.2f}%")
print(f"Md common percentage: {len(ro_md) / len(sentence_numbers_md) * 100:.2f}%")

Ro-Md common: 714
Ro common percentage: 3.58%
Md common percentage: 1.73%


In [80]:
import statistics

# Median length of pos for each language
romana_lengths = [len(sent) for sent in all_pos["romana"]]
moldova_lengths = [len(sent) for sent in all_pos["moldova"]]

print(f"Romana median: {statistics.median(romana_lengths)}")
print(f"Moldova median: {statistics.median(moldova_lengths)}")

# Median length of unique pos for each language
romana_lengths = [len(set(sent)) for sent in all_pos["romana"]]
moldova_lengths = [len(set(sent)) for sent in all_pos["moldova"]]

print(f"Romana unique median: {statistics.median(romana_lengths)}")
print(f"Moldova unique median: {statistics.median(moldova_lengths)}")

# Most common unique pos for each language
romana_unique = []
moldova_unique = []

for sent in all_pos["romana"]:
    romana_unique.extend(set([tag for tag in sent if tag != "PUNCT"]))

for sent in all_pos["moldova"]:
    moldova_unique.extend(set([tag for tag in sent if tag != "PUNCT"]))
    
romana_unique = Counter(romana_unique)
moldova_unique = Counter(moldova_unique)

print(f"Romana unique: {romana_unique.most_common(5)}")
print(f"Moldova unique: {moldova_unique.most_common(5)}")

# Most common unique pos for each language in percentage
romana_total = romana_unique.total()
moldova_total = moldova_unique.total()

romana_unique_percentage = {k: v / romana_total * 100 for k, v in romana_unique.items()}
moldova_unique_percentage = {k: v / moldova_total * 100 for k, v in moldova_unique.items()}

romana_unique_percentage_common = sorted(romana_unique_percentage.items(), key=lambda x: x[1], reverse=True)[:5]
romana_unique_percentage_common_rounded = {}
for k, v in romana_unique_percentage_common:
    romana_unique_percentage_common_rounded[k] = f"{v:.2f}%"
moldova_unique_percentage_common = sorted(moldova_unique_percentage.items(), key=lambda x: x[1], reverse=True)[:5]
moldova_unique_percentage_common_rounded = {}
for k, v in moldova_unique_percentage_common:
    moldova_unique_percentage_common_rounded[k] = f"{v:.2f}%"

print(f"Romana unique percentage: {romana_unique_percentage_common_rounded}")
print(f"Moldova unique percentage: {moldova_unique_percentage_common_rounded}")

Romana median: 19.0
Moldova median: 22.0
Romana unique median: 9.0
Moldova unique median: 9.0
Romana unique: [('NOUN', 21194), ('ADP', 19825), ('AUX', 19630), ('VERB', 17469), ('ADJ', 13602)]
Moldova unique: [('NOUN', 99063), ('AUX', 93100), ('ADP', 91760), ('VERB', 75680), ('ADJ', 73411)]
Romana unique percentage: {'NOUN': '11.97%', 'ADP': '11.19%', 'AUX': '11.08%', 'VERB': '9.86%', 'ADJ': '7.68%'}
Moldova unique percentage: {'NOUN': '12.98%', 'AUX': '12.20%', 'ADP': '12.02%', 'VERB': '9.92%', 'ADJ': '9.62%'}
